# Naive approach 2 - Deep Neural Network

In [1]:
# Get dataset
import torchvision.datasets as datasets
import torchvision.transforms as transforms

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [2]:
import numpy as np

# Define classes
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Flatten images and separate labels
train_images = np.array([np.array(image).flatten() for image, label in trainset])
train_labels = np.array([label for image, label in trainset])
test_images = np.array([np.array(image).flatten() for image, label in testset])
test_labels = np.array([label for image, label in testset])

print(f"train_images shape: {train_images.shape}")
print(f"train_labels shape: {train_labels.shape}")
print(f"test_images shape: {test_images.shape}")
print(f"test_labels shape: {test_labels.shape}")

train_images shape: (50000, 3072)
train_labels shape: (50000,)
test_images shape: (10000, 3072)
test_labels shape: (10000,)


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl

class DeepNeuralNetwork(pl.LightningModule):
    def __init__(self, input_size, num_classes):
        super().__init__()
        self.layer1 = nn.Linear(input_size, 1536)
        self.layer2 = nn.Linear(1536, 768)
        self.layer3 = nn.Linear(768, 384)
        self.layer4 = nn.Linear(384, 96)
        self.layer5 = nn.Linear(96, num_classes)

    def forward(self, x):
        out = torch.relu(self.layer1(x))
        out = torch.relu(self.layer2(out))
        out = torch.relu(self.layer3(out))
        out = torch.relu(self.layer4(out))
        out = self.layer5(out)
        return out

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.CrossEntropyLoss()(y_hat, y)
        preds = torch.argmax(y_hat, dim=1)
        acc = torch.sum(preds == y).item() / (len(y) * 1.0)
        self.log('train_loss', loss)
        self.log('train_acc', acc)
        return loss

    def configure_optimizers(self):
        optimizer = optim.SGD(self.parameters(), lr=0.01)
        return optimizer

# Define input size and number of classes
input_size = train_images.shape[1]
num_classes = len(classes)

# Create PyTorch Lightning trainer
trainer = pl.Trainer(max_epochs=30)

# Create deep neural network model
model = DeepNeuralNetwork(input_size, num_classes)

# Train the model
trainset_tensor = torch.utils.data.TensorDataset(torch.Tensor(train_images), torch.Tensor(train_labels).long())
trainloader = torch.utils.data.DataLoader(trainset_tensor, batch_size=32, shuffle=True)
trainer.fit(model, trainloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs



  | Name   | Type   | Params
----------------------------------
0 | layer1 | Linear | 4.7 M 
1 | layer2 | Linear | 1.2 M 
2 | layer3 | Linear | 295 K 
3 | layer4 | Linear | 37.0 K
4 | layer5 | Linear | 970   
----------------------------------
6.2 M     Trainable params
0         Non-trainable params
6.2 M     Total params
24.935    Total estimated model params size (MB)


Epoch 29: 100%|██████████| 1563/1563 [00:14<00:00, 110.65it/s, v_num=6]

`Trainer.fit` stopped: `max_epochs=30` reached.


Epoch 29: 100%|██████████| 1563/1563 [00:14<00:00, 110.43it/s, v_num=6]


In [14]:
%reload_ext tensorboard
%tensorboard --logdir=lightning_logs/

In [5]:
# Convert test images and labels to PyTorch tensors
test_images_tensor = torch.Tensor(test_images)
test_labels_tensor = torch.Tensor(test_labels).long()

# Get model predictions
with torch.no_grad():
    outputs = model(test_images_tensor)
    _, predicted = torch.max(outputs.data, 1)

# Compute accuracy
total = test_labels_tensor.size(0)
correct = (predicted == test_labels_tensor).sum().item()
accuracy = 100 * correct / total

print('Accuracy: {:.2f}%'.format(accuracy))

Accuracy: 53.33%
